# Test LSSTGhostBuster package

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../src'))
os.environ["OMP_NUM_THREADS"] = "8" # export OMP_NUM_THREADS=8

In [ ]:
from ghost_buster import display_image as display
from ghost_buster import sources_image as sim
from ghost_buster import ghosts_fit as gfit
from ghost_buster import ghosts_simu as gsim

In [ ]:
import pylab as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

import lsst.afw.display as afwDisplay
import lsst.afw.image as afwimage
import lsst.afw.fits as afwfits

In [ ]:
ghost_048=afwimage.ImageF.readFits("ghost_048.fits", 0)

In [ ]:
display.displayImage(ghost_048, title="2024111100048")
display.displayImageGhosts(ghost_048, zmin=400, zmax=800, title="2024111100048")
display.displayImageGhostsBW(ghost_048, title="2024111100048")

In [ ]:
img_048 = ghost_048.getArray()
md_048 = afwfits.readMetadata("ghost_048.fits")
md_048["RA"], md_048["DEC"], md_048["ROTPA"], md_048["FILTBAND"]

In [ ]:
display.displayImageGhostsFlux(img_048, minflux=400, maxflux=800)
display.displayReal(img_048)

In [ ]:
img_048_wtsrc = sim.removeSources(img_048)
display.displayRemoveSources(img_048, img_048_wtsrc)
img_048.shape

In [ ]:
t, _ = gsim.getTransmissionRate("g")
t

In [ ]:
ComCam, wavelength = gsim.initTelescope("g")
Params = gsim.initParams(thetapos=(-0.030448523671382042, 0.008756076320729538), rot=md_048["ROTPA"])

In [ ]:
# x, y ,flux, paths = gsim.batoidCalcul(ComCam, Params, wavelength, debug=True)
x, y, flux = gsim.getGhosts(ComCam, Params, wavelength, nbghost=10, ghostmap=True)

In [ ]:
x, y = gsim.rotAfterBatoid(x, y, md_054["ROTPA"])

In [ ]:
px, py = ghost_054.getDimensions()[0], ghost_048.getDimensions()[1]
Simu = gsim.getSimuImage(px, py, x, y, flux, binning=8)
Simu = gfit.applyGrid(img_048, Simu)

In [ ]:
display.displaySimu(Simu)

In [ ]:
clean, _ = gfit.fitGhosts(img_048, Simu)

In [ ]:
display.displayFit(img_048, Simu, clean)

In [ ]:
print(f"Hist shape : {Simu.shape}\nImage shape : {img_048.shape}")

In [ ]:
xpos, ypos = gsim.thetatopixel((1577, 1586), (-0.030448523671382042, 0.008756076320729538), 8)
xpos, ypos

In [ ]:
_, artefacts = sim.removeSourcesBoth(img_048, clean)
display.displayRemoveSourcesAndArtefacts(img_048_wtsrc, artefacts)
display.displayCut(img_048_wtsrc, xpos, ypos)
display.displayCut(artefacts, xpos, ypos)

In [ ]:
CCDs = gsim.ccd_extractor(img_048)

In [ ]:
display.displayReal(CCDs[4][0])

In [ ]:
yorigin, xorigin = CCDs[4][1]
xpos_sub = xpos - xorigin
ypos_sub = ypos - yorigin

In [ ]:
r, profil = gfit.profil_radial(CCDs[4][0], (xpos_sub, ypos_sub), bin_width=1.0)
plt.plot(r, profil)
plt.xlabel("Rayon (pixels)")
plt.ylabel("Valeur moyenne")
#plt.yscale('log')
plt.xlim(0, 40)
#plt.ylim(900, 1500)
plt.axhline(955.241, color="red")
plt.title(f"Profil radial autour de ( {xpos_sub:.2f}, {ypos_sub:.2f} )")
plt.savefig('profil')
plt.show()

In [ ]:
def power_law_profile(r, A, r0, n, b):
    return A / (1 + (r / r0)**n) + b

r, profil = gfit.profil_radial(CCDs[4][0], (xpos_sub, ypos_sub), bin_width=2.0)
smoothed_profil = gaussian_filter1d(profil, sigma=1)
popt, _ = curve_fit(power_law_profile, r, smoothed_profil, p0=[1e4, 5, 3, 950,])

plt.plot(r, smoothed_profil)
plt.plot(r, power_law_profile(r, *popt))
plt.xlabel("Rayon (pixels)")
plt.ylabel("Valeur moyenne")
#plt.yscale('log')
plt.xlim(0, 40)
#plt.ylim(900, 1500)
#plt.axhline(955.241, color="red")
plt.title(f"Profil radial autour de ( {xpos_sub:.2f}, {ypos_sub:.2f} )")
plt.show()

print(f"I_0 optimal    : {popt[0]:.2f}")
print(f"r_0 optimal    : {popt[1]:.2f}")
print(f"n optimal      : {popt[2]:.2f}")
print(f"offset optimal : {popt[3]:.2f}")